In [1]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [2]:
import os
import torch
from huggingface_hub import HfFolder
from datasets import load_from_disk
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!unzip tatoeba_tokenized.zip

Archive:  tatoeba_tokenized.zip
   creating: tokenized_train_ds/
   creating: tokenized_train_ds/train/
  inflating: tokenized_train_ds/train/dataset_info.json  
  inflating: tokenized_train_ds/train/state.json  
  inflating: tokenized_train_ds/train/data-00000-of-00002.arrow  
  inflating: tokenized_train_ds/train/data-00001-of-00002.arrow  
   creating: tokenized_train_ds/eval/
  inflating: tokenized_train_ds/eval/dataset_info.json  
  inflating: tokenized_train_ds/eval/state.json  
  inflating: tokenized_train_ds/eval/data-00000-of-00001.arrow  


In [4]:
trainset = load_from_disk('tokenized_train_ds/train')
testset = load_from_disk('tokenized_train_ds/eval')

In [5]:
#model_id = "t5-small" # Hugging Face Model Id
model_id = "oliverli0414/oliverli0414-tatoeba_finetuned" # Hugging Face Model Id
tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [6]:
# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100

# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [9]:
batch_size = 128
repository_id = f"{model_id.split('/')[0]}-tatoeba_finetuned"
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    generation_max_length=273,
    weight_decay=0.01,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=False,
    bf16=True,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    #evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    #load_best_model_at_end=True,
    # push to hub parameters
    report_to="tensorboard",
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
    push_to_hub=True,
)

In [10]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset=testset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

<ipython-input-10-ff0810d3198b>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,0.779600
1000,0.769700
1500,0.777200
2000,0.764600
2500,0.767600
3000,0.755900
3500,0.754700
4000,0.756600
4500,0.749700
5000,0.754600


In [15]:
torch.save(model.state_dict(), "tatoeba_model_3.pth")

In [16]:
!zip -r t5-small-tatoeba_finetuned_3.zip t5-small-tatoeba_finetuned

  adding: t5-small-tatoeba_finetuned/ (stored 0%)
  adding: t5-small-tatoeba_finetuned/config.json (deflated 63%)
  adding: t5-small-tatoeba_finetuned/checkpoint-5216/ (stored 0%)
  adding: t5-small-tatoeba_finetuned/checkpoint-5216/scheduler.pt (deflated 56%)
  adding: t5-small-tatoeba_finetuned/checkpoint-5216/config.json (deflated 63%)
  adding: t5-small-tatoeba_finetuned/checkpoint-5216/optimizer.pt (deflated 7%)
  adding: t5-small-tatoeba_finetuned/checkpoint-5216/tokenizer_config.json (deflated 94%)
  adding: t5-small-tatoeba_finetuned/checkpoint-5216/trainer_state.json (deflated 68%)
  adding: t5-small-tatoeba_finetuned/checkpoint-5216/training_args.bin (deflated 52%)
  adding: t5-small-tatoeba_finetuned/checkpoint-5216/generation_config.json (deflated 29%)
  adding: t5-small-tatoeba_finetuned/checkpoint-5216/special_tokens_map.json (deflated 85%)
  adding: t5-small-tatoeba_finetuned/checkpoint-5216/spiece.model (deflated 48%)
  adding: t5-small-tatoeba_finetuned/checkpoint-5216

In [17]:
from google.colab import files
files.download("tatoeba_model_3.pth")
files.download("t5-small-tatoeba_finetuned_3.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
model.load_state_dict(torch.load("tatoeba_model_3.pth"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
text = 'Translate French to English: bonjour'
input_id = tokenizer(text, return_tensors='pt').input_ids
output_id = model.generate(input_id)
print(tokenizer.decode(output_id[0],skip_special_tokens=True))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)